# A few simple data exercises

Here we will use a subset of a dataset sourced from Kaggle: [https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset](https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset)

It consists of anonymized scraped house listings for the whole U.S. for one point in time during 2024.

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql


### Helper function definitions

In the next cell we define some functions which will be useful for this exercise.

In [2]:
def load_listings(filen,dtypes_dict={'brokered_by': str,
                                     'status':str,
                                     'price':float,
                                     'bed':float,
                                     'bath':float,
                                     'acre_lot':float,
                                     'street':str,
                                     'city':str,
                                     'state':str,
                                     'zip_code':str,
                                     'house_size':float,
                                     'prev_sold_date':str
                                    }):
    df_in = pd.read_csv(filen,dtype=dtypes_dict)
    df_in['prev_sold_date'] = pd.to_datetime(df_in.prev_sold_date,errors='coerce')
    return df_in



def refresh_sqlite_table_from_df(dbfile,tablename,df):
    conn = sql.connect(dbfile)
    df.to_sql(tablename, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

def df_from_sqlite_query(dbfile,query):
    conn = sql.connect(dbfile)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df
    





## Load and peek and the dataset

Notice that this make take a few seconds

In [3]:
%%time

listings_df = load_listings('../data/some_listings.csv')

listings_df

CPU times: total: 1.09 s
Wall time: 1.12 s


,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,22505,for_sale,269000.0,NaN,NaN,4.70,148075,Vero Beach,Florida,32966,NaN,2013-12-04
1,67091,sold,112000.0,4.0,1.0,0.22,28055,Neillsville,Wisconsin,54456,1420.0,2022-04-28
2,85045,for_sale,74500.0,NaN,NaN,3.65,1981526,Johnson City,Tennessee,37615,NaN,NaT
3,16829,sold,995000.0,3.0,2.0,0.06,1069629,San Francisco,California,94110,1275.0,2022-04-20
4,16250,for_sale,124900.0,2.0,1.0,0.24,1674940,Garland,Texas,75040,1040.0,1990-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,22439,sold,611000.0,5.0,4.0,0.25,315753,Newberry,Florida,32669,2982.0,2022-02-04
499996,46248,sold,319900.0,2.0,2.0,0.15,193859,Huntley,Illinois,60142,1579.0,2022-03-17
499997,41784,sold,650000.0,4.0,2.0,0.17,925931,Phoenix,Arizona,85086,2350.0,2022-04-12
499998,96692,for_sale,459000.0,NaN,NaN,1.71,669352,Seneca,South Carolina,29672,NaN,2021-05-13


## Put our dataset into a local SQL database file

This allows us to simulate querying an actual SQL server.

In [ ]:
%%time

refresh_sqlite_table_from_df('../data/sql/some_listings.db','listings',listings_df)

## Our first SQL query--pull the first 50 observations off the top

In [ ]:
query = """SELECT
  *
FROM listings
LIMIT 50
"""



df_from_sqlite_query('../data/sql/some_listings.db',
                     query)

## Pandas equivalent

In [ ]:
listings_df.iloc[0:50]

## Now, let's pull the 50 most expensive houses

In [ ]:
query = """SELECT
  *
FROM listings
ORDER BY
  price DESC
LIMIT 50
"""



df_from_sqlite_query('../data/sql/some_listings.db',
                     query)

## 50 most expensive houses, using Pandas

In [ ]:
listings_df.sort_values(by='price',ascending=False).iloc[0:50]

## Which states have better data quality for house size?

We have to multiply the numerator of our "fraction" below by 1.0, because otherwise SQLite will treat the quotient of two integers as an integer and round down.

In [ ]:

query = """SELECT
  state,
  (1.0*COUNT(house_size))/COUNT(*) as fraction_of_house_size_observed,
  COUNT(*) as total_listings
FROM listings
GROUP BY
  state
ORDER BY
  state
"""



df_from_sqlite_query('../data/sql/some_listings.db',
                     query)

## Data coverage for house size by state, with Pandas

In [ ]:
summary_df = listings_df[['state','house_size','status']].groupby('state').count().reset_index()
summary_df['house_size'] /= summary_df['status']
summary_df.columns = ['state','fraction_of_house_size_observed','total_listings']
summary_df

## How many brokers in each state?

In [ ]:

query = """SELECT
  state,
  COUNT(DISTINCT brokered_by) as num_brokers
FROM listings
GROUP BY
  state
ORDER BY
  state
"""



df_from_sqlite_query('../data/sql/some_listings.db',
                     query)

## Count brokers with Pandas

In [ ]:
listings_df[['state','brokered_by']].groupby('state').nunique().reset_index()

## Average house price

Because SQLite does not have a built-in standard deviation function, below we use a subquery to compute the means by state so that we can calculate standard deviations in the final step.

Depending on the version of SQL you are using, there may be other more efficient ways to do this. Also, many versions of SQL have a built-in standard deviation function.

In [ ]:
query = """SELECT
  a.state as state,
  mean_price,
  SQRT(SUM(POWER(price - mean_price,2))/num_price_obs) as stdev_price,
  (1.0*num_price_obs)/COUNT(*) as fraction_of_price_observed,
  COUNT(*) as total_listings
FROM listings as a
LEFT JOIN (SELECT
  state,
  AVG(price) as mean_price,
  COUNT(price) as num_price_obs
FROM listings
GROUP BY
  state) as b
ON a.state = b.state
GROUP BY 
  a.state
ORDER BY
  a.state
"""

df_from_sqlite_query('../data/sql/some_listings.db',
                     query)

## Average house price by state, with Pandas

In [ ]:
listings_df[['state','price','status']].groupby('state').agg({'price':['mean','std'],
                                                              'status':'count'}).reset_index()

## Histogram of house prices

Notice that we can't see much, because the outliers drag out the $x$-axis range

In [ ]:
listings_df.price.hist(bins=100)

### Restricting the $x$-axis range

In [ ]:
listings_df.price.hist(bins=100,range=[0,2000000])

### Plotting log house price

Notice this looks like a much tidier distribution.

We will have to exclude points where the price is zero, because $\lim\limits_{x\to 0} \log x = -\infty$

In [ ]:
listings_df['log_price'] = np.log(listings_df['price'])
listings_df.loc[listings_df.price > 0,'log_price'].hist(bins=100)

## Adding log price to our summarizing query

Notice that the standard deviations of log price are all significantly less than the means--in contrast to the standard deviations of the level of price, which were all bigger than the mean!

This is because the un-logged prices are a very skewed (i.e. asymmetrical) distribution, and the outliers inflate the standard deviation. The distribution of log price, on the other hand, is symmetrical, and the outliers are "tamed" by the log transformation.

In [ ]:
query = """SELECT
  a.state as state,
  mean_price,
  SQRT(SUM(POWER(price - mean_price,2))/num_price_obs) as stdev_price,
  (1.0*num_price_obs)/COUNT(*) as fraction_of_price_observed,
  mean_log_price,
  SQRT(SUM(POWER(LN(price) - mean_log_price,2))/num_log_price_obs) as stdev_log_price,
  (1.0*num_log_price_obs)/COUNT(*) as fraction_of_log_price_observed,
  COUNT(*) as total_listings
FROM listings as a
LEFT JOIN (SELECT
  state,
  AVG(price) as mean_price,
  AVG(LN(price)) as mean_log_price,
  COUNT(price) as num_price_obs,
  SUM(CASE WHEN price > 0 THEN 1 ELSE 0 END) as num_log_price_obs
FROM listings
GROUP BY
  state) as b
ON a.state = b.state
GROUP BY 
  a.state
ORDER BY
  a.state
"""

df_from_sqlite_query('../data/sql/some_listings.db',
                     query)

## Do the same with Pandas

Notice that Pandas does not automatically exclude the non-finite values from the mean calculation.

In [ ]:
listings_df[['state','price','log_price','status']].groupby('state').agg({'price':['mean','std'],
                                                                          'log_price':['mean','std'],
                                                                          'status':'count'}).reset_index()

## Correlation between house size and price

Below we'll calculate the simple correlation (Pearson's correlation coefficient), and make a scatterplot.

In [ ]:
listings_df['log_house_size'] = np.log(listings_df.house_size)

display(listings_df.plot.scatter(x='log_house_size',y='log_price'))

display(listings_df[['log_house_size','log_price']].corr())


## Focus on one state

Now the correlation is higher. Why do you think that is?

Modify the code below to try this for other state, too!

In [ ]:

display(listings_df.loc[listings_df.state == 'Texas'].plot.scatter(x='log_house_size',y='log_price'))

display(listings_df.loc[listings_df.state == 'Texas',['log_house_size','log_price']].corr())

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(listings_df.log_house_size,listings_df.log_price,'.')
plt.show()



## Some more exercises

  1. Calculate price per square foot, and summarize by state.
  2. Construct a regression or other model to predict either house price, or price per square foot, from observable characteristics.

#### Extra credit:
  - Save the dataset as a Parquet file and compare file size + loading time.
  - Commit your changes locally with Git.
  - Create a forked version of the repo on your own Github account, and push your updates.